In [21]:
"""
朴素贝叶斯的实现
朴素贝叶斯的公式
P(类别|多个特征) = P(类别) * P(多个特征|类别) / P(多个特征)
P(多个特征)指的是多个特征的组合出现的概率，在预测时，大家都是一样的，所以不用计算。
P(类别)表示这个类别的先验概率，即在训练集中各个类别本身分布的概率。
P(多个特征|类别)表示给定类别以后，多个特征组合出现的概率，以下面的训练集举例：
evaluation=unacc(不可接受)时，buying price=vhigh, price of the maintenance=vhigh, 
doors=2, persons=2, the size of luggage boot=small, safety=low这样的例子有多少。
由于上述的特征之间是and关系，在实际例子中，可能会出现count=0的情况，朴素贝叶斯认为特征之间无关联关系，可以分开
计算，P(多个特征|类别)退化为：
P(特征1|类别)*P(特征2|类别)...
综上所述，朴素贝叶斯其实就是各种count。

实现其实没有考虑平滑化的问题,训练集中可能存在P(x1|Y)=0的情况。
朴素贝叶斯的缺点在于实际上各个特征肯定存在关联，不应该独立来看，但是简单来看，性能还不错。

"""
df = pd.read_table('../data/car.txt', sep='\s*,\s*')
df.head(5)

car_label = df[["evaluation"]]
car_unique_label = car_label["evaluation"].unique()

df_test = df[len(df) - 10:len(df)]
df_train = df[0:len(df) - 10]


/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:24: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.


,Buying price,price of the maintenance,doors,persons,the size of luggage boot,safety,evaluation
0,vhigh,vhigh,2,2,small,low,unacc
1,vhigh,vhigh,2,2,small,med,unacc
2,vhigh,vhigh,2,2,small,high,unacc
3,vhigh,vhigh,2,2,med,low,unacc
4,vhigh,vhigh,2,2,med,med,unacc


In [30]:
def predict(df_train, df_test, label_name, car_unique_label):
    predict = []
    total = len(df_train)
    total = len(df_train)
    feature_cols = df_train.columns.values.tolist()
    feature_cols.remove("evaluation")
    feature_indexes = np.arange(0, len(feature_cols))
    for i in range(len(df_test)):
        class_prob = []
        for class_label in car_unique_label:
            df_train_with_class_label = df_train[df_train[label_name] == class_label]
            total_with_class = len(df_train_with_class_label)
            class_label_prior = float(total_with_class) / float(total)
            # 类别的先验概率
            class_probability = class_label_prior
            for feature, feature_index in zip(feature_cols, feature_indexes):
                # 计算各个feature的P(特征|类别)
                # 取sample的每个属性值
                test_sample_feature_value = df_test[i:i+1].values.tolist()[0][feature_index]
                df_feature = df_train_with_class_label[df_train_with_class_label[feature] == test_sample_feature_value]
                total_with_class_and_value = len(df_feature)
                feature_value_in_class_probability = float(total_with_class_and_value) / float(total_with_class)
                class_probability *= feature_value_in_class_probability
            # 计算完了所有的feature
            class_prob.append((class_label, class_probability))
        class_sort = sorted(class_prob, key=lambda x: x[1], reverse=True)
        predict.append(class_sort[0][0])
    return predict

def accracy(predict, y):
    total = len(predict)
    correct = 0
    for i in range(0, len(predict)):
        if predict[i] == y[i]:
            correct +=1
    return correct / total

array(['unacc', 'acc', 'vgood', 'good'], dtype=object)

In [ ]:
predict = predict(df_train, df_test, "evaluation", car_unique_label)
test_y = df_test["evaluation"].values.tolist()
print(accracy(predict, test_y))
